<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML321ENSkillsNetwork32585014-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Collaborative Filtering based Recommender System using Non-negative Matrix Factorization**


Estimated time needed: **60** minutes


In the previous lab, we have performed KNN on user-item interaction matrix to estimate the rating of unknown items based on the aggregation of the user's K nearest neighbor's ratings. Finding nearest neighbors are based on similarity measurements among users or items with big similarity matrices.


The KNN algorithm is memory-based which means we need to keep all instances for prediction and maintain a big similarity matrix. These can be infeasible if our user/item scale is large, for example, 1 million users will require a 1 million by 1 million similarity matrix, which is very hard to load into RAM for most computation environments.


#### Non-negative matrix factorization


In the machine learning course, you have learned a dimensionality reduction algorithm called Non-negative matrix factorization (NMF), which decomposes a big sparse matrix into two smaller and dense matrices.

Non-negative matrix factorization can be one solution to big matrix issues. The main idea is to decompose the big and sparse user-interaction into two smaller dense matrices, one represents the transformed user features and another represents the transformed item features.


An example is shown below, suppose we have a user-item interaction matrix $A$ with 10000 users and 100 items (10000 x 100), and its element `(j, k)` represents the rating of item `k` from user `j`. Then we could decompose $A$ into two smaller and dense matrices $U$ (10000 x 16) and $I$ (16 x 100). for user matrix $U$, each row vector is a transformed latent feature vector of a user, and for the item matrix $I$, each column is a transformed latent feature vector of an item.

Here the dimension 16 is a hyperparameter defines the size of the hidden user and item features, which means now the shape of transposed user feature vector and item feature vector is now 16 x 1.


The magic here is when we multiply the row `j` of $U$ and column `k` of matrix $I$, we can get an estimation to the original rating $\hat{r}\_{jk}$.

For example, if we preform the dot product user ones  row vector in $U$ and item ones  column vector in $I$, we can get the rating estimation of user one to item one, which is the element (1, 1) in the original interaction matrix $I$. This r


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/module\_4/images/nmf.png)


Note $I$ is short for Items, and it is not an identity matrix.


Then how do we figure out the values in $U$ and $I$ exactly? Like many other machine learning processes, we could start by initializing the values of $U$ and $I$, then define the following distance or cost function to be minimized:


$$\sum_{r_{jk} \in {train}} \left(r_{jk} - \hat{r}_{jk} \right)^2,$$


where $\hat{r}_{ij}$ is the dot product of $u_j^T$ and $i_k$:


$$\hat{r}_{jk} = u_j^Ti_k$$


The cost function can be optimized using stochastic gradient descent (SGD) or other optimization algorithms, just like in training the weights in a logistic regression model (there are several additional steps so the matrices have no negative elements) .


## Objectives


After completing this lab you will be able to:


*   Perform NMF-based collaborative filtering on the user-item matrix


***


### Load and exploring dataset


Let's first load our dataset, i.e., the user-item (learn-course) interaction matrix


In [1]:
import pandas as pd

In [2]:
rating_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/ratings.csv"
rating_df = pd.read_csv(rating_url)

In [3]:
rating_df.head()

,user,item,rating
0,1889878,CC0101EN,3.0
1,1342067,CL0101EN,3.0
2,1990814,ML0120ENv3,3.0
3,380098,BD0211EN,3.0
4,779563,DS0101EN,3.0


The dataset contains three columns, `user id`, `item id`, and `the rating`. Note that this matrix is presented as the dense or vertical form, you may convert it using `pivot` to the original sparse matrix:


In [9]:
rating_sparse_df = rating_df.pivot(index='user', columns='item', values='rating').fillna(0).reset_index().rename_axis(index=None, columns=None)
rating_sparse_df.head()

,user,AI0111EN,BC0101EN,BC0201EN,BC0202EN,BD0101EN,BD0111EN,BD0115EN,BD0121EN,BD0123EN,...,SW0201EN,TA0105,TA0105EN,TA0106EN,TMP0101EN,TMP0105EN,TMP0106,TMP107,WA0101EN,WA0103EN
0,2,0.0,3.0,0.0,0.0,3.0,2.0,0.0,2.0,2.0,...,0.0,2.0,0.0,3.0,0.0,2.0,2.0,0.0,3.0,0.0
1,4,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,...,0.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,2.0
2,5,2.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,...,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0
3,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next, you need to implement NMF-based collaborative filtering, and you may choose one of the two following implementation options:

*   The first one is to use `Surprise` which is a popular and easy-to-use Python recommendation system library.
*   The second way is to implement it with `numpy`, `pandas`, and `sklearn`. You may need to write a lot of low-level implementation code along the way.


In [4]:
from sklearn.decomposition import NMF
import numpy as np
from sklearn.metrics import mean_squared_error

course_url = "course_ratings.csv"
course_ds = pd.read_csv(course_url)
#Formato sparse
rating_sparse_ds = course_ds.pivot(index='user', columns='item', values='rating').fillna(0).reset_index().rename_axis(index=None, columns=None)
rating_sparse_ds.head()

model = NMF(n_components=100, init='random', random_state=0)
W = model.fit_transform(X_train)
H = model.components_

FileNotFoundError: [Errno 2] No such file or directory: 'course_ratings.csv'

In [ ]:
course_ds.head()

In [20]:
rating_sparse_ds.shape

(33901, 127)

In [24]:
W.shape

(22713, 100)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test= train_test_split(rating_sparse_ds, test_size=0.33, random_state=42)

In [41]:
prediction = model.inverse_transform(model.transform(X_test))
from sklearn.metrics import mean_squared_error
mean_squared_error(X_test, prediction)

0.011730927686459388

In [25]:
H.shape

(100, 127)

In [26]:
W.shape

(22713, 100)

In [27]:
model.transform(X_test).shape

(11188, 100)

In [28]:
X_test.shape

(11188, 127)

In [35]:
np.argsort(model.transform(X_test)[0])

array([49, 69, 67, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52,
       51, 50, 98, 70, 71, 72, 74, 97, 95, 94, 93, 92, 91, 89, 88, 87, 48,
       86, 83, 82, 81, 80, 79, 78, 77, 76, 75, 85, 47, 99, 45, 24,  7, 22,
       46, 21, 20, 19, 18,  8, 16, 15, 14, 13, 12, 11, 25, 27, 26, 36, 40,
       39, 38, 37,  5, 35, 41, 34,  2, 31, 30,  3,  1, 44, 33, 42,  9, 66,
       68, 84, 10, 73, 90, 43, 29, 32,  6, 23, 17, 28,  4, 96,  0])

## Summary


In this lab, you have learned and practiced NMF-based collaborative filtering. The basic idea is to decompose the original user-item interaction matrix into two smaller and dense user and item matrices. Then, we have built the two matrices, we can easily estimate the unknown ratings via the dot product of specific row in user matrix and specific column in item matrix.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML321ENSkillsNetwork32585014-2022-01-01)


### Other Contributors


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-10-25        | 1.0     | Yan        | Created the initial version |


Copyright © 2022 IBM Corporation. All rights reserved.
